In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image

In [3]:
# Adds images to array
images = np.zeros((2000, 250, 250, 3)) # I changed this to be the actual dimensions that np.asarray will copy into
j = 0
for i in range(1522): # Our ranges were off by 1 so I fixed that
  try:
    image_name = 'F_' + str(i) + '.jpg'
    path = "Fire/" + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [5]:
for i in range(1011): # Range was off by 1 like above
  try:
    image_name = 'NF_' + str(i) + '.jpg'
    path = 'Non_Fire/' + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [6]:
targets = np.zeros(2000)
for i in range(1000):
  targets[i] = 1

In [7]:
# This will make the images array 2d so it can be better converted into a dataframe and csv
images = images.reshape((2000, 187500))
df = pd.DataFrame(images)
df['target'] = targets

In [8]:
# Shuffles df so we can split into test/train
df = df.sample(frac = 1)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,187491,187492,187493,187494,187495,187496,187497,187498,187499,target
1508,123.0,151.0,199.0,123.0,151.0,199.0,123.0,151.0,199.0,123.0,...,33.0,26.0,18.0,20.0,13.0,7.0,68.0,61.0,55.0,0.0
1264,125.0,176.0,179.0,127.0,178.0,181.0,125.0,176.0,180.0,121.0,...,8.0,49.0,51.0,8.0,50.0,49.0,11.0,53.0,52.0,0.0
627,217.0,66.0,37.0,218.0,67.0,38.0,221.0,70.0,41.0,226.0,...,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,1.0
571,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
446,51.0,27.0,15.0,46.0,22.0,10.0,42.0,18.0,6.0,43.0,...,149.0,51.0,6.0,154.0,61.0,28.0,173.0,83.0,56.0,1.0


In [37]:
train_data = df[:1500]
test_data = df[1500:]

In [45]:
train_images = np.asarray(train_data)
train_targets = train_images[: ,-1]
train_images_minus_targets = np.delete(train_images, -1, axis=1)

test_images = np.asarray(test_data)
test_targets = test_images[: ,-1]
test_images_minus_targets = np.delete(test_images, -1, axis=1)

In [46]:
train_images_minus_targets = np.reshape(train_images_minus_targets, (1500, 250, 250, 3))
test_images_minus_targets = np.reshape(test_images_minus_targets, (500, 250, 250, 3))

In [40]:
model = models.Sequential()
model.add(layers.Conv2D(250, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(500, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(500, (3, 3), activation='relu'))

C:\Users\eliza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 248, 248, 250)       │           7,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 124, 124, 250)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 122, 122, 500)       │       1,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 61, 61, 500)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 59, 59, 500)         │       2,250,500 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,383,000 (12.91 MB)

 Trainable params: 3,383,000 (12.91 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
model.add(layers.Flatten())
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(2))

In [43]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 248, 248, 250)       │           7,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 124, 124, 250)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 122, 122, 500)       │       1,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 61, 61, 500)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 59, 59, 500)         │       2,250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1740500)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 500)                 │     870,250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │           1,002 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 873,634,502 (3.25 GB)

 Trainable params: 873,634,502 (3.25 GB)

 Non-trainable params: 0 (0.00 B)

In [47]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images_minus_targets, train_targets, epochs=10, 
                    validation_data=(test_images_minus_targets, test_targets))

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/sequential_1_1/dense_4_1/MatMul/MatMul_1 defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[1740500,500] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node StatefulPartitionedCall/gradient_tape/sequential_1_1/dense_4_1/MatMul/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_2209]